In [3]:
import json
import requests
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from time import sleep
import time
from bs4 import BeautifulSoup
import re

In [5]:
URL = "https://api.hh.ru/vacancies"

In [7]:
job_titles_list = ["data scientist", "data analyst"]

In [9]:
base_params_template = {
    'area': 113,           # Регион: Россия
    'per_page': 100,
    'order_by': 'publication_time'
}

In [11]:
all_vacancies_data = []

In [13]:
MAX_PAGES = 4

In [15]:
def download_vacancy(base_params, page_num):
    """
    Выполняет HTTP-запрос к API HH.ru с обновлением номера страницы.
    """
    # КОПИРУЕМ базовый шаблон и обновляем номер страницы
    current_params = base_params.copy()
    current_params["page"] = page_num

    try:
        response = requests.get(URL, params=current_params)
        response.raise_for_status() 
        return response.json()
    except requests.exceptions.RequestException as e:
        print(f"Ошибка HTTP на странице {page_num}: {e}")
        return None
    except json.JSONDecodeError:
        print(f"Ошибка декодирования JSON на странице {page_num}")
        return None

In [17]:
def get_full_details(vacancy_id):
    """
    Получает полное описание вакансии по ее ID.
    """
    detail_url = f"https://api.hh.ru/vacancies/{vacancy_id}"
    try:
        response = requests.get(detail_url)
        response.raise_for_status()
        return response.json()
    except requests.exceptions.RequestException as e:
        print(f"Ошибка получения деталей для ID {vacancy_id}: {e}")
        return None

In [23]:
# 3. ФУНКЦИЯ ОБРАБОТКИ ДАННЫХ (Сплющивание)
def process_vacancy(vacancy, search_term):
    salary_data = vacancy.get('salary')
    min_salary = None
    max_salary = None
    if salary_data and isinstance(salary_data, dict) and salary_data.get('from') and salary_data.get('to'):
        min_salary = salary_data['from']
        max_salary = salary_data['to']

    # 2. Работодатель: вложенное поле
    employer_info = vacancy.get('employer', {})
    employer_name = employer_info.get('name', 'N/A')

    # 3. Город: вложенное поле
    area_info = vacancy.get('area', {})
    city_name = area_info.get('name', 'N/A')

    # 4. Требования и обязанности (извлекаем чистый текст)
    snippet = vacancy.get('snippet', {})
    requirement = snippet.get('requirement', '')
    responsibility = snippet.get('responsibility', '')

    # 5. Опыт: вложенное поле
    experience_info = vacancy.get('experience', {})
    experience_name = experience_info.get('name', 'N/A')

    # 1. Получаем ID вакансии
    vacancy_id = vacancy.get('id')
    full_description = "N/A"

    if vacancy_id:
        # 2. Делаем новый запрос за деталями
        full_data = get_full_details(vacancy_id)

        # 3. Извлекаем поле 'description' (там HTML-код)
        if full_data and full_data.get('description'):
            full_description = full_data['description']

        time.sleep(0.1)

    # Создание плоского словаря
    flat_row = {
        'full_description': full_description,
        'search_term': search_term, # Какую должность мы искали
        'vacancy_id': vacancy.get('id'),
        'vacancy_name': vacancy.get('name'),
        'city_name': city_name,
        'min_salary': min_salary,
        'max_salary': max_salary,
        'employer_name': employer_name,
        'published_at': vacancy.get('published_at'),
        'experience': experience_name,
        'schedule': vacancy.get('schedule', {}).get('name'),
        'employment': vacancy.get('employment', {}).get('name'),
        'requirement': requirement,
        'responsibility': responsibility,
        # Добавьте другие поля по необходимости
    }
    return flat_row

In [25]:
# 4. ОСНОВНОЙ ЦИКЛ ПАРСИНГА
for job in job_titles_list:
    print(f"--- Начинаем парсинг для: '{job}' ---")

    # 1. Определяем параметры для текущей профессии (добавляем 'text')
    current_search_params = base_params_template.copy()
    current_search_params['text'] = job

    for page in range(MAX_PAGES):
        vac_data = download_vacancy(current_search_params, page)

        # Если загрузка не удалась (вернула None или пустой словарь)
        if not vac_data:
            print(f"Пропуск страницы {page} из-за ошибки запроса.")
            continue # Переходим к следующей итерации цикла

        # БЕЗОПАСНАЯ ПРОВЕРКА (устраняет KeyError, как мы обсуждали)
        if 'items' in vac_data:
            vacancies_on_page = vac_data['items']

            if not vacancies_on_page:
                print(f"Страница {page} пуста. Завершаем сбор для '{job}'.")
                break # Выходим из цикла по страницам, так как вакансии закончились

            for vacancy in vacancies_on_page:
                # 2. Сплющиваем и добавляем плоский словарь в финальный список
                flat_row = process_vacancy(vacancy, job)
                all_vacancies_data.append(flat_row)

            print(f"Обработано {len(vacancies_on_page)} вакансий на странице {page}. Всего: {len(all_vacancies_data)}")

        else:
            print(f"Ошибка структуры данных на странице {page}: Нет ключа 'items'.")

        time.sleep(0.2) # Пауза между запросами

--- Начинаем парсинг для: 'data scientist' ---
Обработано 100 вакансий на странице 0. Всего: 100
Обработано 100 вакансий на странице 1. Всего: 200
Обработано 100 вакансий на странице 2. Всего: 300
Обработано 8 вакансий на странице 3. Всего: 308
--- Начинаем парсинг для: 'data analyst' ---
Обработано 100 вакансий на странице 0. Всего: 408
Обработано 100 вакансий на странице 1. Всего: 508
Обработано 100 вакансий на странице 2. Всего: 608
Обработано 100 вакансий на странице 3. Всего: 708


In [27]:
# 5. СОЗДАНИЕ DATAFRAME И CSV
print("\n--- Завершение сбора данных ---")
print(f"Общее количество собранных записей: {len(all_vacancies_data)}")

if all_vacancies_data:
    df = pd.DataFrame(all_vacancies_data)
    df['min_salary'] = df['min_salary'].fillna(0)
    df['max_salary'] = df['max_salary'].fillna(0)

    filename = 'hh_vacancies_data.csv'
    df.to_csv(filename, index=False, encoding='utf-8')
    print(f"\n✅ Датасет успешно создан и сохранен в файл: {filename}")
else:
    print("❌ Список вакансий пуст. Файл CSV не создан.")


--- Завершение сбора данных ---
Общее количество собранных записей: 708

✅ Датасет успешно создан и сохранен в файл: hh_vacancies_data.csv


In [31]:
df.head()

,full_description,search_term,vacancy_id,vacancy_name,city_name,min_salary,max_salary,employer_name,published_at,experience,schedule,employment,requirement,responsibility
0,<p><strong>1221Systems </strong>– аккредитован...,data scientist,127757615,Junior Data Scientist,Нижний Новгород,0.0,0.0,1221Системс,2025-11-17T08:26:25+0300,От 1 года до 3 лет,Удаленная работа,Полная занятость,Проектный опыт работы с временными рядами или ...,Time Series Forecasting (ритейл: прогноз прода...
1,<p><strong><em>МЫ ПРЕДЛАГАЕМ:</em></strong></p...,data scientist,127757227,Ведущий специалист отдела математического моде...,Москва,0.0,0.0,Eqvanta,2025-11-17T08:19:50+0300,От 1 года до 3 лет,Удаленная работа,Полная занятость,Опыт в разработке моделей: опыт работы в облас...,Разработка и совершенствование моделей: создан...
2,<p><strong>Мы предлагаем:​​​​​</strong></p> <u...,data scientist,127592071,Data Scientist,Новосибирск,0.0,0.0,Финтехсофт,2025-11-17T06:25:12+0300,От 1 года до 3 лет,Полный день,Полная занятость,Дополнительные сертификаты в области <highligh...,Разработка и внедрение прогнозных моделей. Соз...
3,<p>Если ТЕБЕ нравятся крутые и амбициозные зад...,data scientist,127117611,Аналитик данных,Нижний Новгород,0.0,0.0,КСК-Эйч Ар,2025-11-16T16:55:15+0300,Нет опыта,Удаленная работа,Полная занятость,Расширение технических знаний и развитие из ан...,Автоматизация выгрузки и сбора данных. - Ad/ho...
4,<p>Если ТЕБЕ нравятся крутые и амбициозные зад...,data scientist,127117612,Аналитик данных,Самара,0.0,0.0,КСК-Эйч Ар,2025-11-16T16:55:15+0300,Нет опыта,Удаленная работа,Полная занятость,Расширение технических знаний и развитие из ан...,Автоматизация выгрузки и сбора данных. - Ad/ho...


In [33]:
df['employer_name'].value_counts()

employer_name
СБЕР                                                      53
Ozon                                                      22
билайн                                                    18
WILDBERRIES                                               12
Русский Фонд Недвижимости                                 11
                                                          ..
Livornica                                                  1
Headshot                                                   1
Красногорсклексредства                                     1
Интерактивное агентство Это Легко                          1
ГАУК Московское агентство организации отдыха и туризма     1
Name: count, Length: 396, dtype: int64

In [35]:
df['full_description'][0]

'<p><strong>1221Systems </strong>– аккредитованная IT-компания, специализирующаяся на разработке IT-решений для бизнеса розничной и оптовой торговли.</p> <p>Наша компания выполняет разработку IT-продуктов группы компании &quot;Сладкая жизнь&quot; – одного из крупнейших дистрибьюторов продуктов питания в России.</p> <p>На данный момент находимся в поиске <strong>Data Scientist</strong> на проекты по разработке корпоративных систем для наших партнеров – брендов SPAR, EUROSPAR. Проекты направлены на прогнозирование спроса и потребительской активности с целью оптимизации закупочной деятельности и управления остатками товаров на складах.</p> <p>Ищем универсального бойца, который может как развить проект со стадии PoC, так и распространить текущие наработки на другие компании по существующему пайплайну (есть возможность прокачать MLOps навыки).</p> <p><strong>Чем предстоит заниматься:</strong></p> <ul> <li>Time Series Forecasting (ритейл: прогноз продаж, прогноз количества заказов);</li> <li

In [37]:
#число уникальных описаний
unique_description = df['full_description'].nunique()
unique_description

651

In [39]:
unique_employers = df['employer_name'].nunique()
total_vacancies = len(df)
print(f"Общее количество вакансий: {total_vacancies}")
print(f"Количество уникальных работодателей: {unique_employers}")

Общее количество вакансий: 708
Количество уникальных работодателей: 396


In [41]:
top_employers = df['employer_name'].value_counts().head(10)
print("\nТоп-10 работодателей:")
print(top_employers)


Топ-10 работодателей:
employer_name
СБЕР                         53
Ozon                         22
билайн                       18
WILDBERRIES                  12
Русский Фонд Недвижимости    11
МТС                          10
Альфа-Банк                   10
Т-Банк                        8
X5 Tech                       8
Aston                         7
Name: count, dtype: int64


In [43]:
df_alt = df.copy()

In [46]:
df_alt.notnull().sum()

full_description    708
search_term         708
vacancy_id          708
vacancy_name        708
city_name           708
min_salary          708
max_salary          708
employer_name       708
published_at        708
experience          708
schedule            708
employment          708
requirement         706
responsibility      706
dtype: int64

In [ ]:
#df_alt = df_alt.loc[:, df_alt.notnull().sum() >= treshhold]
#df_alt.info()

Очищу датасет до минимума необходимой информации

In [54]:
df_alt.fillna('', inplace=True)

очищу данные от html символов, пунктуации и тд

In [57]:
import nltk
import unicodedata

from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

In [59]:
nltk.download('stopwords')
nltk.download('punkt_tab')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [62]:
#nlp_ru = stanza.Pipeline(lang='ru', processors='tokenize,lemma')
russian_stopwords = set(stopwords.words('russian'));

In [64]:
def clean_text_simple(input_text, stop_words_set=None):
    
    if pd.isna(input_text) or input_text is None:
        return ""

    text = str(input_text)

    # 1. Удаляем HTML
    text = re.sub(r'<[^>]+>', ' ', text)

    # 2. Удаляем URL
    text = re.sub(r'https?://\S+|www\.\S+', ' ', text)

    # 3. Сохраняем базовую структуру текста
    text = re.sub(r'[^a-zA-Zа-яА-ЯёЁ0-9\s\-_\.]', ' ', text)

    # 4. Нижний регистр
    text = text.lower()

    # 5. Обрабатываем составные слова
    text = re.sub(r'(\b\w+)-(\w+\b)', r'\1_\2', text)

    # 6. Токенизация
    words = text.split()

    # 7. Фильтрация
    if stop_words_set:
        filtered_words = [
            word for word in words
            if (word not in stop_words_set and
                len(word) > 1 and
                not word.isdigit())  # убираем чистые числа
        ]
    else:
        filtered_words = [word for word in words if len(word) > 1 and not word.isdigit()]

    return ' '.join(filtered_words)

In [66]:
text = 'О компании Мы — успешный fashion-бренд с 15-летней историей и собственным производством женской одежды. Полный цикл: от закупки ткани до продажи на маркетплейсах. Наш основной оборот идёт через Wildberries и Ozon (входим в топ-1% продавцов одежды на Ozon ). Сейчас мы запускаем проект по построению аналитической/рекомендательной платформы с нуля . Цель — сделать управление ассортиментом, ценами, закупками, производством и рекламой максимально прозрачным и основанным на данных . Если внутренняя система покажет результат, мы планируем вывести её на рынок как отдельный SaaS-продукт для других продавцов маркетплейсов. Это возможность присоединиться на старте и влиять на архитектуру продукта.(уже есть заинтересованные селлеры) Задачи 1. Сбор данных и интеграции Подключение API Wildberries, Ozon и других площадок. Получение данных: Продажи (выручка, заказы, выкупы, возвраты, статусы). Остатки на складах, логистика. Рекламные метрики (показы, клики, CTR, CPC, CPA, ROI). Цены, динамика скидок, участие в акциях. Позиции в поиске, отзывы, рейтинг. Интеграция данных из 1С (по ODATA) . Загрузка данных из Excel/CSV. Парсинг маркетплейсов для конкурентного анализа. 2. Построение хранилища данных Проектирование и реализация Data LakeHouse . Хранение данных в S3 или аналогах , первичная обработка. Создание структуры данных (сырые → очищенные → витрины). Использование ClickHouse или других колоночных БД. Настройка базовых ETL/ELT-процессов . 3. Аналитика и прогнозирование Прогноз спроса и продаж (по товарам и категориям). Оптимизация остатков и распределение поставок по складам WB/Ozon . Автоматизация рекламных кампаний: динамические ставки, удаление неэффективных ключей и кластеров, анализ CTR/конверсий. Расчёт маржинальности и прибыли. Оценка эффективности SKU, поставок и логистики. 4. Визуализация и отчётность Построение дашбордов в Yandex DataLens . Создание таблиц и аналитических отчётов. Возможна разработка части аналитики в самописном веб-приложении . Требования Опыт работы с данными: ETL, SQL, Python . Знание API: работа с REST API маркетплейсов, интеграция с 1С ODATA. Опыт с БД: PostgreSQL, ClickHouse (или аналогами). Инструменты: Docker, Git . Библиотеки: pandas, requests, airflow (или опыт других пайплайн-менеджеров). BI: DataLens (обязательно), понимание метрик аналитики. Умение строить прогнозы (time series, ML — плюс). Умение самостоятельно доводить задачи до результата. Плюсом будет Опыт проектирования Data LakeHouse. Опыт работы с маркетплейсами (WB/Ozon). Знание MLOps. Опыт работы в e-commerce или производстве. Навыки FastAPI/Flask для интеграций и сервисов. Мы предлагаем Участие в проекте с нуля — ключевая роль. Возможность влиять на архитектуру и решения. Рост вместе с продуктом. Гибкий график, гибридный формат (после ИС — частично удалёнка). Реальная свобода выбора технологий и решений. Долгосрочная работа над продуктом, а не «таск-менеджмент». Важно — перед откликом Мы ищем самостоятельного специалиста , а не стажёра или начинающего уровня. На старте у нас нет ресурсов учить с нуля , поэтому: ❗ Если вы не умеете или не готовы быстро научиться: работать с API WB/Ozon, ETL, Python, работать с ClickHouse/PostgreSQL, строить дашборды в DataLens(или других системах) — — не откликайтесь на вакансию. В сопроводительном письме укажите: Ваш уровень по навыкам: Python / SQL / API / ClickHouse / DataLens (по 10-балльной шкале). Реальный опыт: 2–3 проекта или задачи, которыми вы гордитесь. Желательный уровень дохода и формат работы (офис/гибрид). — на испытательный срок мы ищем сотрудников исключительно в офис!!! , не откликайтесь на вакансию если это вас не устраивает! p.s.: Мы не хотим делать копию существующих на рынке продуктов, мы тестировали многие, но они не отвечают нашим задачам и задачам селлеров с кем мы знакомы, у нас уже есть некоторые наработки, поэтому это скорее гибрид нескольких продуктов, чтобы максимально автоматизировать рутинные задачи и быстро принимать управленческие решения.'

In [68]:
cleaned_text = clean_text_simple(text,  russian_stopwords)

In [70]:
print(cleaned_text)

компании успешный fashion_бренд 15_летней историей собственным производством женской одежды. полный цикл закупки ткани продажи маркетплейсах. наш основной оборот идёт wildberries ozon входим топ_1 продавцов одежды ozon запускаем проект построению аналитической рекомендательной платформы нуля цель сделать управление ассортиментом ценами закупками производством рекламой максимально прозрачным основанным данных внутренняя система покажет результат планируем вывести её рынок отдельный saas_продукт других продавцов маркетплейсов. это возможность присоединиться старте влиять архитектуру продукта. заинтересованные селлеры задачи 1. сбор данных интеграции подключение api wildberries ozon других площадок. получение данных продажи выручка заказы выкупы возвраты статусы остатки складах логистика. рекламные метрики показы клики ctr cpc cpa roi цены динамика скидок участие акциях. позиции поиске отзывы рейтинг. интеграция данных 1с odata загрузка данных excel csv. парсинг маркетплейсов конкурентног

Обработаю текстовые фичи функцией для удаления html, пунктуации, приведения кстандартному виду

In [73]:
df_alt['description_cleaned'] = df_alt['full_description'].apply(
    lambda text: clean_text_simple(text, russian_stopwords)
)

In [75]:
df_alt['requirement_cleaned'] = df_alt['requirement'].apply(
    lambda text: clean_text_simple(text, russian_stopwords))

In [77]:
df_alt['requirement_cleaned'][1]

'опыт разработке моделей опыт работы области разработки скоринговых моделей года. аналитические способности умение эффективно работать с...'

In [133]:
df_final = df_alt.copy()

In [135]:
df_final = df_final.drop(['full_description', 'requirement'], axis=1)

In [137]:
df_final.head()

,search_term,vacancy_id,vacancy_name,city_name,min_salary,max_salary,employer_name,published_at,experience,schedule,employment,responsibility,description_cleaned,requirement_cleaned
0,data scientist,127757615,Junior Data Scientist,Нижний Новгород,0.0,0.0,1221Системс,2025-11-17T08:26:25+0300,От 1 года до 3 лет,Удаленная работа,Полная занятость,Time Series Forecasting (ритейл: прогноз прода...,1221systems аккредитованная it_компания специа...,проектный опыт работы временными рядами рядами...
1,data scientist,127757227,Ведущий специалист отдела математического моде...,Москва,0.0,0.0,Eqvanta,2025-11-17T08:19:50+0300,От 1 года до 3 лет,Удаленная работа,Полная занятость,Разработка и совершенствование моделей: создан...,предлагаем стабильность компания финансовом ры...,опыт разработке моделей опыт работы области ра...
2,data scientist,127592071,Data Scientist,Новосибирск,0.0,0.0,Финтехсофт,2025-11-17T06:25:12+0300,От 1 года до 3 лет,Полный день,Полная занятость,Разработка и внедрение прогнозных моделей. Соз...,предлагаем официальное трудоустройство тд бела...,дополнительные сертификаты области data scienc...
3,data scientist,127117611,Аналитик данных,Нижний Новгород,0.0,0.0,КСК-Эйч Ар,2025-11-16T16:55:15+0300,Нет опыта,Удаленная работа,Полная занятость,Автоматизация выгрузки и сбора данных. - Ad/ho...,тебе нравятся крутые амбициозные задачи мечтае...,расширение технических знаний развитие аналити...
4,data scientist,127117612,Аналитик данных,Самара,0.0,0.0,КСК-Эйч Ар,2025-11-16T16:55:15+0300,Нет опыта,Удаленная работа,Полная занятость,Автоматизация выгрузки и сбора данных. - Ad/ho...,тебе нравятся крутые амбициозные задачи мечтае...,расширение технических знаний развитие аналити...


In [140]:
df_final['meta_header'] = (
    'Вакансия: ' + df_final['vacancy_name'].fillna('') + '.'
    ' Город:  ' + df_final['city_name'].fillna('') + '.' +
    " Опыт: " + df_final['experience'].fillna('') + ". " +
    "График: " + df_final['schedule'].fillna('') + ". " +
    "Занятость: " + df_final['employment'].fillna('') + ". "
    "Минимальная зарплата: " + df_final['min_salary'].fillna('').astype(str) + "."
    "Максимальная зарплата: " + df_final['max_salary'].fillna('').astype(str) + "."
)


In [142]:
df_requirements = df_final.copy()
df_requirements['rag_chunk'] = (df_requirements['meta_header'] +
                                'Требования: ' + df_requirements['requirement_cleaned'].fillna('требования не указаны')+ '.'
                               " Обязанности: " + df_requirements['responsibility'].fillna('Обязанности не указаны.'))

In [144]:
df_requirements.head()

,search_term,vacancy_id,vacancy_name,city_name,min_salary,max_salary,employer_name,published_at,experience,schedule,employment,responsibility,description_cleaned,requirement_cleaned,meta_header,rag_chunk
0,data scientist,127757615,Junior Data Scientist,Нижний Новгород,0.0,0.0,1221Системс,2025-11-17T08:26:25+0300,От 1 года до 3 лет,Удаленная работа,Полная занятость,Time Series Forecasting (ритейл: прогноз прода...,1221systems аккредитованная it_компания специа...,проектный опыт работы временными рядами рядами...,Вакансия: Junior Data Scientist. Город: Нижни...,Вакансия: Junior Data Scientist. Город: Нижни...
1,data scientist,127757227,Ведущий специалист отдела математического моде...,Москва,0.0,0.0,Eqvanta,2025-11-17T08:19:50+0300,От 1 года до 3 лет,Удаленная работа,Полная занятость,Разработка и совершенствование моделей: создан...,предлагаем стабильность компания финансовом ры...,опыт разработке моделей опыт работы области ра...,Вакансия: Ведущий специалист отдела математиче...,Вакансия: Ведущий специалист отдела математиче...
2,data scientist,127592071,Data Scientist,Новосибирск,0.0,0.0,Финтехсофт,2025-11-17T06:25:12+0300,От 1 года до 3 лет,Полный день,Полная занятость,Разработка и внедрение прогнозных моделей. Соз...,предлагаем официальное трудоустройство тд бела...,дополнительные сертификаты области data scienc...,Вакансия: Data Scientist. Город: Новосибирск....,Вакансия: Data Scientist. Город: Новосибирск....
3,data scientist,127117611,Аналитик данных,Нижний Новгород,0.0,0.0,КСК-Эйч Ар,2025-11-16T16:55:15+0300,Нет опыта,Удаленная работа,Полная занятость,Автоматизация выгрузки и сбора данных. - Ad/ho...,тебе нравятся крутые амбициозные задачи мечтае...,расширение технических знаний развитие аналити...,Вакансия: Аналитик данных. Город: Нижний Новг...,Вакансия: Аналитик данных. Город: Нижний Новг...
4,data scientist,127117612,Аналитик данных,Самара,0.0,0.0,КСК-Эйч Ар,2025-11-16T16:55:15+0300,Нет опыта,Удаленная работа,Полная занятость,Автоматизация выгрузки и сбора данных. - Ad/ho...,тебе нравятся крутые амбициозные задачи мечтае...,расширение технических знаний развитие аналити...,Вакансия: Аналитик данных. Город: Самара. Опы...,Вакансия: Аналитик данных. Город: Самара. Опы...


In [156]:
rag_chunk_coloumn = df_requirements['rag_chunk']
city_name_coloumn = df_requirements['city_name']
vacancy_name_coloumn = df_requirements['vacancy_name']
df_final = pd.concat([df_final, rag_chunk_coloumn, city_name_coloumn, vacancy_name_coloumn], axis=1)

In [158]:
df_requirements['rag_chunk'][0]

'Вакансия: Junior Data Scientist. Город:  Нижний Новгород. Опыт: От 1 года до 3 лет. График: Удаленная работа. Занятость: Полная занятость. Минимальная зарплата: 0.0.Максимальная зарплата: 0.0.Требования: проектный опыт работы временными рядами рядами регрессии года. владение python sql. навыки работы sklearn.... Обязанности: Time Series Forecasting (ритейл: прогноз продаж, прогноз количества заказов). Анализировать и визуализировать данные, выявлять закономерности, представлять результаты. Предлагать и проводить...'

In [162]:
df_requirements = df_requirements[['vacancy_id', 'vacancy_name', 'city_name', 'rag_chunk',  'description_cleaned']] 

In [164]:
df_requirements.head(10)

,vacancy_id,vacancy_name,city_name,rag_chunk,description_cleaned
0,127757615,Junior Data Scientist,Нижний Новгород,Вакансия: Junior Data Scientist. Город: Нижни...,1221systems аккредитованная it_компания специа...
1,127757227,Ведущий специалист отдела математического моде...,Москва,Вакансия: Ведущий специалист отдела математиче...,предлагаем стабильность компания финансовом ры...
2,127592071,Data Scientist,Новосибирск,Вакансия: Data Scientist. Город: Новосибирск....,предлагаем официальное трудоустройство тд бела...
3,127117611,Аналитик данных,Нижний Новгород,Вакансия: Аналитик данных. Город: Нижний Новг...,тебе нравятся крутые амбициозные задачи мечтае...
4,127117612,Аналитик данных,Самара,Вакансия: Аналитик данных. Город: Самара. Опы...,тебе нравятся крутые амбициозные задачи мечтае...
5,126737674,Старший бизнес-аналитик CLTV,Москва,Вакансия: Старший бизнес-аналитик CLTV. Город:...,наша команда каждый день работает повышением к...
6,127401542,Data Scientist,Москва,Вакансия: Data Scientist. Город: Москва. Опыт...,проекте нейроллаб исследовательская нейромарке...
7,126718438,Data Scientist (Junior),Москва,Вакансия: Data Scientist (Junior). Город: Мос...,bnd консалтинговая компания специализирующаяся...
8,125416786,Senior Data Scientist (NLP),Санкт-Петербург,Вакансия: Senior Data Scientist (NLP). Город: ...,привет ищем ds команду research and developmen...
9,127080680,Data Scientist / ML Engineer,Санкт-Петербург,Вакансия: Data Scientist / ML Engineer. Город:...,цифровые корпоративные технологии ведущий подр...


In [182]:
city = df_requirements[(df_requirements['city_name'] == 'Санкт-Петербург')  (df_requirements['vacancy_name'] == 'Data Scientist')]
city

,vacancy_id,vacancy_name,city_name,rag_chunk,description_cleaned
2,127592071,Data Scientist,Новосибирск,Вакансия: Data Scientist. Город: Новосибирск....,предлагаем официальное трудоустройство тд бела...
6,127401542,Data Scientist,Москва,Вакансия: Data Scientist. Город: Москва. Опыт...,проекте нейроллаб исследовательская нейромарке...
8,125416786,Senior Data Scientist (NLP),Санкт-Петербург,Вакансия: Senior Data Scientist (NLP). Город: ...,привет ищем ds команду research and developmen...
9,127080680,Data Scientist / ML Engineer,Санкт-Петербург,Вакансия: Data Scientist / ML Engineer. Город:...,цифровые корпоративные технологии ведущий подр...
10,127432787,Data Scientist,Краснодар,Вакансия: Data Scientist. Город: Краснодар. О...,dataacquisition команда экспертов области big ...
...,...,...,...,...,...
670,126731632,Супервайзер по мерчендайзингу,Санкт-Петербург,Вакансия: Супервайзер по мерчендайзингу. Город...,ищем супервайзера мерчендайзингу юг_юго-запад ...
674,124010098,Team Lead продуктовой аналитики,Санкт-Петербург,Вакансия: Team Lead продуктовой аналитики. Гор...,несколько лет назад yclients амбициозным старт...
681,127646077,Delivery Manager 1C / Руководитель 1С,Санкт-Петербург,Вакансия: Delivery Manager 1C / Руководитель 1...,кадровое агентство brightman находится поиске ...
692,127636612,Менеджер по маркетингу и PR,Санкт-Петербург,Вакансия: Менеджер по маркетингу и PR. Город: ...,гкс альянс это многопрофильный холдинг объедин...


In [188]:
# 5. СОЗДАНИЕ Финального DATAFRAME И CSV
print("\nЗавершение обработки данных")

filename = 'hh_vacancies_data_final.csv'

df_requirements.to_csv(filename, index=False, encoding='utf-8')
print(f"\n✅ Датасет успешно создан и сохранен в файл: {filename}")



Завершение обработки данных

✅ Датасет успешно создан и сохранен в файл: hh_vacancies_data_final.csv
